# Convolutional Neural Network (CNN) in PyTorch
Sebbene esistano alternative, come la libreria `Keras`, la strategia migliore per scrivere modelli in `PyTorch` è sviluppare sotto classi di una classe che rappresenta __moduli generici__. 
I moduli possono contenere altri moduli, permettendo di annidare elementi funzionali. Le operazioni più comuni, come layer lineari, convoluzionali o self attention etc, sono tutti moduli in PyTorch.

In generale, il pattern di programmazione di un modulo consiste di due fasi: 
- Nella __prima fase__ si dichiarano i moduli necessari alla creazione del modulo corrente, viene fatto nel costruttore. Si scelgono le dimensioni dei sotto moduli e viene effettuata l'inizializzazione.
- Nella __seconda fase__ si va a sovraccaricare il metodo `foreward()`, che, data un'istanza, ritorna un nuovo tensore risultate dalle operazioni specificate nel modulo.

Le dimensioni dei layer non vengono calcolate automaticamente in PyTorch, a differenza di Keras. Quindi, per evitare errori, è utile disegnare uno __schema del modello__.

<div align="center">
<img src="img_CNN.png" width="350">
</div>

Nella figura mostrata abbiamo 4 __blocchi convoluzionali__ tutti uguali tra loro, i quali vanno in un __flattern__ per poi proseguire in una __classification head__. 
La scelta di appiattire l'ultima feature map perchè, lavorando con galassie, l'oggetto da classificare è quasi sempre al centro. Inoltre, il __collo di bottiglia__ permetterà di ridurre il numero di pesi nella classification head. 
Nel disegno sono presenti valori numerici, rappresentano i valori delle dimensioni dei tensori in ingresso e uscita dei blocchi. La strategia dei blocchi è di raddoppiare le feature maps (primo numero della tripletta), dimezzando la dimensione (gli altri due numeri).

Il __blocco__ è una sequenza di __conv 2D__, __batchnorm__, __ReLu__ e per finire __max pooling 2D__ che dimezza la dimensione. Dato che si tratta di una sequenza posso usare `nn.Sequential()` come contenitore. 

In [1]:
import torch 
from torch import nn

In [6]:
class Block(nn.Module):
    def __init__(self, in_channels, num_filters, kernel_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels = in_channels,
                        out_channels = num_filters,
                        kernel_size = kernel_size,
                        padding = "same"),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(in_channels = num_filters, # ingresso uguale a out del layer precedente
                        out_channels = num_filters, 
                        kernel_size = kernel_size,
                        padding = "same"),  
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2, 2)) # dimezza altezza e larghezza delle immagini
        )
        
    def forward(self, x):
        return self.layers(x)

Il blocco è stato definito, ora posso iniziare a costruire la classe principale.

I primi 4 blocchi, quelli di colore rosso nella figura, sono raggruppati in un `Sequential()`. Da notare che il numero di filtri raddoppia ogni volt, mentre la forma delle feature map si riduce di 1/4.

Il __bottleneck__ è un layer convoluzionale con kernel 1x1, che riduce il numero di feature maps a quello di partenza.
Le feature map vengono appiattite in un vettore lungo 6272 dal __flattern__.

Infine, il __multi layer perceptron__ è un layer lineare con 128 neuroni e attivazione ReLu e forma la __classification head__.


In [3]:
class SimpleCNN(nn.Module):
    def __init__(self, num_filters, mpl_size, num_classes):
        super().__init__()
        self.blocks = nn.Sequential(
            Block(3, num_filters, (5, 5)), 
            Block(num_filters, num_filters*2, (3, 3)),
            Block(num_filters*2, num_filters*4, (3, 3)),
            Block(num_filters*4, num_filters*8, (3, 3))
        )
        
        self.bottle_neck = nn.Conv2d(
            in_channels = num_filters*8,
            out_channels = num_filters,
            kernel_size = (1, 1)
        )
        
        self.flatten = nn.Flatten()
        
        self.mlp = nn.Sequential(
            nn.Linear(in_features = 14*14*num_filters, out_features = mpl_size),
            nn.ReLU()
        )
        
        self.classification_head = nn.Linear(in_features = mpl_size, out_features = num_classes)
        
    def forward(self, x):
        x = self.blocks(x)
        x = self.bottle_neck(x)
        x = self.flatten(x)
        x = self.mlp(x)
        return self.classification_head(x)

In [10]:
import torchsummary

batch_size = 4
input_size = 224
num_classes = 10
num_filters = 32
mpl_size = 128

input_tensor = torch.rand(batch_size, 3, input_size, input_size)

model = SimpleCNN(num_filters, mpl_size, num_classes)

torchsummary.summary(model, input_size=(3, input_size, input_size), device="cpu")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]           2,432
       BatchNorm2d-2         [-1, 32, 224, 224]              64
              ReLU-3         [-1, 32, 224, 224]               0
            Conv2d-4         [-1, 32, 224, 224]          25,632
       BatchNorm2d-5         [-1, 32, 224, 224]              64
              ReLU-6         [-1, 32, 224, 224]               0
         MaxPool2d-7         [-1, 32, 112, 112]               0
             Block-8         [-1, 32, 112, 112]               0
            Conv2d-9         [-1, 64, 112, 112]          18,496
      BatchNorm2d-10         [-1, 64, 112, 112]             128
             ReLU-11         [-1, 64, 112, 112]               0
           Conv2d-12         [-1, 64, 112, 112]          36,928
      BatchNorm2d-13         [-1, 64, 112, 112]             128
             ReLU-14         [-1, 64, 1

Vediamo che il modello è stato costruito correttamente. Ora posso passare alla fase di training usando il dataset `Galaxy10`.

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader 

import numpy as np
import tables

class GalaxyDataset(Dataset):
    def __init__(self, opts, filename, crop = True):
        self.opts = opts
        h5 = tables.File(filename) # apre il file HDF5
        X = h5.root.images[:] # carica le immagini
        y = h5.root.ans[:] # carica le etichette
        
        X = (X/255).astype(np.float32) # normalizza le immagini
        self.X = torch.tensor(X.transpose(0, 3, 1, 2)) # trasforma le immagini in tensori
        
        if crop: 
            margin = (256 - 224) // 2 # calcola il margine
            self.X = self.X[:,:,margin:-margin,margin:-margin] # ritaglia le immagini
            
        self.num_classes = len(np.unique(y)) # calcola il numero di classi
        EYE = np.eye(self.num_classes) # crea una matrice identità
        self.y = torch.tensor(EYE[y]) # trasforma le matrice identità in tensori
        self.data_shape = X[0].shape # salva la forma delle immagini
        
    def __len__(self):
        return self.X.shape[0] # restituisce il numero di esempi
    
    def __getitem__(self, i):
        return self.X[i], self.y[i] # restituisce l'i-esimo esempio        
    
class MakeDataLoaders():
    def __init__(self, opts, data):
        generator = torch.Generator().manual_seed(opts.seed) # inizializza il generatore di numeri casuali
        train, test = torch.utils.data.random_split(data,
                                                    lengths = [1-opts.test_size, opts.test_size],
                                                    generator = generator) # divide il dataset in training e test set
        
        self.train = DataLoader(train, batch_size = opts.batch_size, shuffle = True) # crea il DataLoader per il training set
        self.test = DataLoader(test, batch_size = opts.batch_size, shuffle = True) # crea il DataLoader per il test set
        

from types import SimpleNamespace
from ipdb import launch_ipdb_on_exception
from helpers import plot

opts = SimpleNamespace()
opts.seed = 1234
opts.test_size = 0.2
opts.batch_size = 128


with launch_ipdb_on_exception():
    data = GalaxyDataset(opts, 'Galaxy10_DECals.h5')
    loaders = MakeDataLoaders(opts, data)
    
# Preparazione del modello
num_filters = 32  # Numero di filtri iniziali
mpl_size = 256  # Dimensione del MLP
num_classes = data.num_classes  # Numero di classi dal dataset

# Inizializza il modello
model = SimpleCNN(num_filters=num_filters, mpl_size=mpl_size, num_classes=num_classes)

# Imposta il dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Modello inviato al dispositivo: {device}")

# Funzione di perdita (accetta one-hot encoding)
criterion = nn.CrossEntropyLoss()

# Ottimizzatore
learning_rate = 0.0002
weight_decay = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Funzione di training
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.argmax(dim=1).to(device)  # Decodifica da one-hot a indice

        optimizer.zero_grad()  # Reset dei gradienti

        outputs = model(inputs)  # Forward pass

        loss = criterion(outputs, labels)  # Calcolo della perdita
        loss.backward()  # Backward pass
        optimizer.step()  # Aggiornamento dei pesi

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += (preds == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples
    return epoch_loss, epoch_accuracy

# Funzione di valutazione
def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.argmax(dim=1).to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples
    return epoch_loss, epoch_accuracy

# Ciclo di training
num_epochs = 100

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(1, num_epochs + 1):
    train_loss, train_accuracy = train_one_epoch(model, loaders.train, criterion, optimizer, device)
    val_loss, val_accuracy = evaluate(model, loaders.test, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}/{num_epochs}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Salvataggio del modello
torch.save(model.state_dict(), "simple_cnn_galaxy10.pth")
print("Modello salvato come 'simple_cnn_galaxy10.pth'")

# Visualizzazione delle curve di perdita e accuratezza
import matplotlib.pyplot as plt

# Plot delle curve di perdita
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot delle curve di accuratezza
plt.figure(figsize=(12, 6))
plt.plot(train_accuracies, label="Training Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


Modello inviato al dispositivo: cpu


KeyboardInterrupt: 